In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers, optimizers
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt

In [22]:
X_raw = np.load(r'C:\Users\lnwwr\Desktop\CS3244\Project\Processed data\x_train.npy')
y_raw = np.load(r'C:\Users\lnwwr\Desktop\CS3244\Project\Processed data\y_train.npy')

Pick first 1000 examples and normalize:

In [23]:
X = tf.cast(X_raw[:1000], dtype=tf.float32) / 255.
y = tf.cast(y_raw[:1000], dtype=tf.int32)

In [24]:
print(X.shape)
print(y.shape)

(1000, 256, 256, 3)
(1000,)


In [37]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X_raw[:1000], y_raw[:1000], stratify=y, test_size=.2, random_state=0)
x_train = tf.cast(x_train, dtype=tf.float32) / 255.
x_test = tf.cast(x_test, dtype=tf.float32) / 255.


In [38]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis()

In [42]:
nsamples_train, nx, ny, nz = x_train.shape
x_train_r = x_train[:,:,:,0].numpy().reshape((nsamples_train,nx*ny))
x_train_g = x_train[:,:,:,1].numpy().reshape((nsamples_train,nx*ny))
x_train_b = x_train[:,:,:,1].numpy().reshape((nsamples_train,nx*ny))

(800, 256, 256)
(800, 65536)


In [56]:
nsamples_test, nx, ny, nz = x_test.shape
x_test_r = x_test[:,:,:,0].numpy().reshape((nsamples_test,nx*ny))
x_test_g = x_test[:,:,:,1].numpy().reshape((nsamples_test,nx*ny))
x_test_b = x_test[:,:,:,1].numpy().reshape((nsamples_test,nx*ny))

In [44]:
X_lda_r = lda.fit(x_train_r,y_train).transform(x_train_r)
X_lda_g = lda.fit(x_train_g,y_train).transform(x_train_g)
X_lda_b = lda.fit(x_train_b,y_train).transform(x_train_b)

In [52]:
print(X_lda_r.shape)
print(X_lda_g.shape)
print(X_lda_b.shape)
print(y_train.shape)

(800, 8)
(800, 8)
(800, 8)
(800,)


In [57]:
X_test_lda_r = lda.fit(x_test_r,y_test).transform(x_test_r)
X_test_lda_g = lda.fit(x_test_g,y_test).transform(x_test_g)
X_test_lda_b = lda.fit(x_test_b,y_test).transform(x_test_b)

In [58]:
print(X_test_lda_r.shape)
print(X_test_lda_g.shape)
print(X_test_lda_b.shape)
print(y_test.shape)

(200, 8)
(200, 8)
(200, 8)
(200,)


Do binary classification: class 0 (Chinese Apple) vs other classes.

In [67]:
y_train_new = []
for label in y_train:
    if label != 0:
        y_train_new.append(1)
    else:
        y_train_new.append(label)
y_train_new = np.asarray(y_train_new)

In [71]:
y_test_new = []
for label in y_test:
    if label != 0:
        y_test_new.append(1)
    else:
        y_test_new.append(label)
y_test_new = np.asarray(y_test_new)

In [74]:
x_train_lda = np.concatenate((X_lda_r, X_lda_g, X_lda_b), axis=1)
x_test_lda = np.concatenate((X_test_lda_r, X_test_lda_g, X_test_lda_b), axis=1)

In [76]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=6)
knn.fit(x_train_lda, y_train_new)
y_pred = knn.predict(x_test_lda)
train_acc = knn.score(x_train_lda, y_train_new)
test_acc = knn.score(x_test_lda, y_test_new)
print("KNN training accuracy: %f" %(train_acc))
print("KNN test accuracy: %f" %(test_acc))

KNN training accuracy: 0.973750
KNN test accuracy: 0.960000


In [ ]:
train_data_generator = ImageDataGenerator(
    rescale=1. / 255,
    fill_mode="constant",
    shear_range=0.2,
    zoom_range=(0.5, 1),
    horizontal_flip=True,
    rotation_range=360,
    channel_shift_range=25,
    brightness_range=(0.75, 1.25))